<a href="https://colab.research.google.com/github/Devraj02-sys/AI-ML-project/blob/main/Face_Rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install keras-facenet
import cv2
import numpy as np
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle
import os

# Load FaceNet model
embedder = FaceNet()

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Dataset path
data_path = "/content/dataset/"

def preprocess_image(image):
    image = cv2.resize(image, (160, 160))
    image = np.expand_dims(image, axis=0)
    return image

def extract_features():
    embeddings = []
    labels = []
    for person in os.listdir(data_path):
        person_path = os.path.join(data_path, person)
        if os.path.isdir(person_path):
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                image = cv2.imread(image_path)
                # Check if image was loaded correctly
                if image is None:
                    print(f"Error loading image: {image_path}")
                    continue  # Skip to the next image
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.1, 4)
                # Check if any faces were detected
                if len(faces) == 0:
                    print(f"No faces detected in image: {image_path}")
                    continue  # Skip to the next image
                for (x, y, w, h) in faces:
                    face = image[y:y+h, x:x+w]
                    face = preprocess_image(face)
                    embedding = embedder.embeddings(face)
                    embeddings.append(embedding[0])
                    labels.append(person)
    # Check if any embeddings were extracted
    if not embeddings:
        print("Error: No embeddings extracted. Check your dataset and face detection settings.")
        return np.empty((0, 0)), np.empty((0,)) # Return empty arrays with the correct dimensions to prevent errors.
    return np.array(embeddings), np.array(labels)

# Extract features and train classifier
print("Extracting features...")
X, y = extract_features()

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train SVM classifier
classifier = SVC(kernel='linear', probability=True)
classifier.fit(X, y_encoded)

# Save model
with open("face_recognition_model.pkl", "wb") as f:
    pickle.dump((classifier, label_encoder), f)

print("Model trained and saved.")

Extracting features...
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━

In [45]:
import cv2
import numpy as np
import pickle
from keras_facenet import FaceNet

# Load trained model
with open("face_recognition_model.pkl", "rb") as f:
    classifier, label_encoder = pickle.load(f)

# Load FaceNet
embedder = FaceNet()

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load test image
image_path = "img1.jpg"  # Change this to your test image path
image = cv2.imread("/content/dataset/Person1/img1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

if len(faces) == 0:
    print("No face detected.")
else:
    (x, y, w, h) = faces[0]
    face = image[y:y+h, x:x+w]
    face = cv2.resize(face, (160, 160))
    face = np.expand_dims(face, axis=0)

    # Extract embedding and predict
    embedding = embedder.embeddings(face)
    prediction = classifier.predict(embedding)
    predicted_label = label_encoder.inverse_transform(prediction)

    print(f"Recognized Person: {predicted_label[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Recognized Person: Person1
